In [ ]:
from lsst.daf.butler import Butler
from lsst.obs.base import DefineVisitsTask, Instrument


In [ ]:
# I think we're talking about ComCam data, right?
# This collections list would work at NCSA, but I suspect it will need adjustment
# elsewhere. Needs to include both the raws and the `camera` dataset.
butler = Butler(
    "/repo/LSSTComCam/",
    collections=["LSSTComCam/raw/all"],
    writeable=True
)


In [ ]:
intra_exposure_id = 2022051300009
extra_exposure_id = 2022051300010

In [ ]:
# Replace "???" below by a query string that corresponds to all wavefront observations.
exposure_data_ids = set(
    butler.registry.queryDataIds(
        [
            "exposure"
        ], 
        where=f"exposure IN ({intra_exposure_id}, {extra_exposure_id})"
    )
)

In [ ]:
exposure_data_ids

In [ ]:
instrument = Instrument.fromName("LSSTComCam", registry=butler.registry)

In [ ]:
config = DefineVisitsTask.ConfigClass()

In [ ]:
instrument.applyConfigOverrides(DefineVisitsTask._DefaultName, config)

In [ ]:
config.groupExposures.name = "one-to-one"

In [ ]:
task = DefineVisitsTask(config=config, butler=butler)

In [ ]:
task.run(exposure_data_ids)

In [ ]:
data_id = {'instrument': 'LSSTComCam', 'detector': 0, 'exposure': 2022012800009}

In [ ]:
metadata = butler.get('raw.visitInfo', dataId=data_id, collections=["LSSTComCam/raw/all"])
print(metadata)